In [1]:
#pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24
Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install python-Levenshtein

  Running setup.py clean for python-Levenshtein
Failed to build python-Levenshtein
    Running setup.py install for python-Levenshtein: started
    Running setup.py install for python-Levenshtein: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Asus\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Asus\\AppData\\Local\\Temp\\pip-install-qtg1x_6z\\python-levenshtein_3df2be91f28d4a5e9348568b8ff9150f\\setup.py'"'"'; __file__='"'"'C:\\Users\\Asus\\AppData\\Local\\Temp\\pip-install-qtg1x_6z\\python-levenshtein_3df2be91f28d4a5e9348568b8ff9150f\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Asus\AppData\Local\Temp\pip-wheel-02l6u9c_'
       cwd: C:\Users\Asus\AppData\Local\Temp\pip-install-qtg1x_6z\python-levenshtein_3df2be91f28d4a5e9348568b8ff9150f\
  Complete output (28 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating b

In [3]:
import gensim
import pandas as pd

# Reading and Exploring the Dataset

The dataset we are using here is a subset of Amazon reviews from the Sports & Outdoors category. The data is stored as a JSON file and can be read using pandas.

Link to the Dataset: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz

In [5]:
df = pd.read_json("Sports_and_Outdoors_5.json", lines=True )
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,5,Woks very good,1390694400,"01 26, 2014"
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,5,Works as well as the factory tool,1328140800,"02 2, 2012"
2,AB2W04NI4OEAD,1881509818,J. Fernald,"[2, 2]",If you don't have a 3/32 punch or would like t...,4,"It's a punch, that's all.",1330387200,"02 28, 2012"
3,A148SVSWKTJKU6,1881509818,"Jusitn A. Watts ""Maverick9614""","[0, 0]",This works no better than any 3/32 punch you w...,4,It's a punch with a Glock logo.,1328400000,"02 5, 2012"
4,AAAWJ6LW9WMOO,1881509818,Material Man,"[0, 0]",I purchased this thinking maybe I need a speci...,4,"Ok,tool does what a regular punch does.",1366675200,"04 23, 2013"


In [6]:
df.shape

(296337, 9)

In [8]:
df.reviewText[0]

'This came in on time and I am veru happy with it, I haved used it already and it makes taking out the pins in my glock 32 very easy'

In [9]:
gensim.utils.simple_preprocess(df.reviewText[0])

['this',
 'came',
 'in',
 'on',
 'time',
 'and',
 'am',
 'veru',
 'happy',
 'with',
 'it',
 'haved',
 'used',
 'it',
 'already',
 'and',
 'it',
 'makes',
 'taking',
 'out',
 'the',
 'pins',
 'in',
 'my',
 'glock',
 'very',
 'easy']

**Simple Preprocessing & Tokenization**

In [12]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

0         [this, came, in, on, time, and, am, veru, happ...
1         [had, factory, glock, tool, that, was, using, ...
2         [if, you, don, have, punch, or, would, like, t...
3         [this, works, no, better, than, any, punch, yo...
4         [purchased, this, thinking, maybe, need, speci...
                                ...                        
296332    [this, is, water, bottle, done, right, it, is,...
296333    [if, you, re, looking, for, an, insulated, wat...
296334    [this, hydracentials, sporty, oz, double, insu...
296335    [as, usual, received, this, item, free, in, ex...
296336    [hydracentials, insulated, oz, water, bottle, ...
Name: reviewText, Length: 296337, dtype: object

# Training the Word2Vec Model

Train the model for reviews. Use a window of size 10 i.e. 10 words before the present word and 10 words ahead. A sentence with at least 2 words should only be considered, configure this using min_count parameter.

Workers define how many CPU threads to be used.

***Initialize the model***

In [15]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)

***Build Vocabulary***

In [17]:
model.build_vocab(review_text, progress_per=1000)

In [18]:
model.epochs

5

In [19]:
model.corpus_count

296337

***Train the Word2Vec Model***

In [30]:
model.train(review_text, total_examples=model.corpus_count, epochs=10)

(182673668, 242993070)

***Finding Similar Words and Similarity between words***

In [31]:
model.wv.most_similar("awful")

[('horrible', 0.710131049156189),
 ('terrible', 0.703340470790863),
 ('ugly', 0.6179715991020203),
 ('unpleasant', 0.5847840905189514),
 ('horrendous', 0.5513478517532349),
 ('funny', 0.5495386719703674),
 ('insane', 0.5412077903747559),
 ('utter', 0.5386292338371277),
 ('crappy', 0.5385554432868958),
 ('overpowering', 0.5292441248893738)]

In [32]:
model.wv.similarity(w1="good", w2="great")

0.79461133

In [33]:
model.wv.similarity(w1="slow", w2="steady")

0.3891777

In [34]:
model.wv.similarity(w1="great", w2="awesome")

0.7145194

In [35]:
model.wv.similarity(w1="great", w2="iphone")

-0.04578647